In [ ]:
from docplex.mp.model import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from numpy.random import choice as np_choice
import numpy as np
import pandas as pd
import time
#------------------------------------------Solve 100 TSP instances using CPLEX-------------------------------------------
def solve_tsp_cplex(distance_matrix):
    n = len(distance_matrix)                  #n = 30
    mdl = Model(name="TSP_CPLEX")
    x = [[mdl.binary_var(name=f"x_{i}_{j}") for j in range(n)] for i in range(n)]              # Decision variables
    u = [mdl.continuous_var(name=f"u_{i}", lb=1, ub=n) for i in range(n)]                      
    mdl.minimize(mdl.sum(distance_matrix[i][j] * x[i][j] for i in range(n) for j in range(n) if i != j)) # Objective function
    # Constraints: Each city must have exactly one incoming and one outgoing edge
    for j in range(n):
        mdl.add_constraint(mdl.sum(x[i][j] for i in range(n) if i != j) == 1)
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[i][j] for j in range(n) if i != j) == 1)   
    for i in range(1, n):                                                                     # Subtour elimination (MTZ formulation)
        for j in range(1, n):
            if i != j:
                mdl.add_constraint(u[i] - u[j] + n * x[i][j] <= n - 1)
    solution = mdl.solve()                                                                    # Solve model
    if solution:
        print("\nDecision Variable Values (x[i][j])")
        for i in range(n):
            for j in range(n):
                if i != j:
                    value = x[i][j].solution_value  
                    print(f"x[{i}][{j}] = {value:.2f}")  
    if solution:
        # Extract the optimal route 
        optimal_edges = [(i, j) for i in range(n) for j in range(n) if i != j and x[i][j].solution_value > 0.5]
        return optimal_edges, solution.objective_value
    else:
        return [], float('inf')
#-----------------------------create 100 instances with 30 cities-------------------------------------------------------
random = np.random
random.seed(1)  
num_instances = 100 
n_small_cities = 30  
instance_results = []
for instance in range(num_instances):
    distance_matrix = np.random.rand(n_small_cities, n_small_cities) * 100             # Generate a random distance matrix
    np.fill_diagonal(distance_matrix, 0)
    distance_matrix = np.round(distance_matrix, 2)     
    optimal_edges, cost = solve_tsp_cplex(distance_matrix)                             # Solve the TSP problem using CPLEX
    instance_results.append((instance + 1, optimal_edges, cost, distance_matrix)) 
    print(f"\nProblem {instance + 1}:")
    print(f"Optimal Edges: {optimal_edges}")
    print(f"Total Cost: {cost:.2f}")
print("\n--- Distance Matrix for Instance 1 ---")
distance_matrix_instance_1 = instance_results[0][3]  
print(distance_matrix_instance_1)
#-----------------------------------------Label edges as 1/-1---------------------------------------------------------
data =[]
for instance_id, optimal_edges, cost, distance_matrix in instance_results:              # generate labeled edges
    n = len(distance_matrix)
    optimal_set = set(optimal_edges)                                                    #quick lookup
    for i in range(n):
        for j in range(n):
            if i != j:
                edge = (i, j)   
                if edge in optimal_set:                                                  # Assign labels
                    label = 1  # Optimal path
                else:
                    label = -1  # Non-optimal path
                data.append((instance_id, edge, label))
df_label_edges = pd.DataFrame(data, columns=["Instance", "Edge", "label"])
#-----------------------------------Compute f1, f2, f3, f4 scores for all edges----------------------------------------------
def compute_graph_features(instance_num, distance_matrix):
    n = len(distance_matrix)            
    f1_values, f2_values, f3_values, f4_values = {}, {}, {}, {}
    for i in range(n):
        min_cik = np.min(distance_matrix[i, :])
        max_cik = np.max(distance_matrix[i, :])
        avg_cik = np.mean(distance_matrix[i, :])
        for j in range(n):  
            if i != j:
                key = (instance_num, (i, j))
                if max_cik - min_cik != 0:
                    f1_values[key] = np.round((distance_matrix[i, j] - min_cik) / (max_cik - min_cik),3)
                else:
                    f1_values[key] = 0
    for j in range(n):
        min_c_kj = np.min(distance_matrix[:, j])
        max_c_kj = np.max(distance_matrix[:, j])
        avg_c_kj = np.mean(distance_matrix[:, j])
        for i in range(n):  
            if i != j:
                key = (instance_num, (i, j))
                if max_c_kj - min_c_kj != 0:
                    f2_values[key] = np.round((distance_matrix[i, j] - min_c_kj) / (max_c_kj - min_c_kj),3)
                    f3_values[key] = np.round((distance_matrix[i, j] - avg_cik) / (max_c_kj - min_c_kj),3)
                    f4_values[key] = np.round((distance_matrix[i, j] - avg_c_kj) / (max_c_kj - min_c_kj),3)
                else:
                    f2_values[key] = f3_values[key] = f4_values[key] = 0
    return f1_values, f2_values, f3_values, f4_values
f1_scores, f2_scores, f3_scores, f4_scores = {}, {}, {}, {}                                  # Compute features for all instances
for instance_num, _, _, distance_matrix in instance_results:
    f1, f2, f3, f4 = compute_graph_features(instance_num, distance_matrix)
    f1_scores.update(f1)
    f2_scores.update(f2)
    f3_scores.update(f3)
    f4_scores.update(f4)
f1_df = pd.DataFrame(f1_scores.items(), columns=["Instance_Edge", "f1"])
f2_df = pd.DataFrame(f2_scores.items(), columns=["Instance_Edge", "f2"])
f3_df = pd.DataFrame(f3_scores.items(), columns=["Instance_Edge", "f3"])
f4_df = pd.DataFrame(f4_scores.items(), columns=["Instance_Edge", "f4"])
for df in [f1_df, f2_df, f3_df, f4_df]:                                                    # Extract instance and edge columns
    df[['Instance', 'Edge']] = pd.DataFrame(df['Instance_Edge'].tolist(), index=df.index)
    df.drop(columns=["Instance_Edge"], inplace=True)
final_features_df = f1_df.merge(f2_df, on=['Instance', 'Edge'])                            # Merge all feature DataFrames into one
final_features_df = final_features_df.merge(f3_df, on=['Instance', 'Edge'])
final_features_df = final_features_df.merge(f4_df, on=['Instance', 'Edge'])                # Merge feature dataset with edge labels
combined_df = final_features_df.merge(df_label_edges, on=['Instance', 'Edge'], how='inner')
combined_df.to_csv("Desktop/features_TSP.csv", index=False)
#------------------------------------------Train the SVM model---------------------------------------------------
df = pd.read_csv("Desktop/features_TSP.csv")                                              # Extract features and labels                                                            
X = df[['f1', 'f2', 'f3', 'f4']]                                                          # Feature columns
y = df['label']                                                                           # Target labels 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
svm_1 = SVC(kernel='linear', probability=True, random_state=42)
svm_1.fit(X_train, y_train)
y_pred = svm_1.predict(X_test)                                       
accuracy = accuracy_score(y_test, y_pred)                                                 # Evaluate the model
print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
#----------------------------------------------penalize misclassifying----------------------------------------
n = len(y_train)                                                                          # Compute class weights
n_pos = (y_train == 1).sum()                                                              # Number of positive points
n_neg = (y_train == -1).sum()                                                             # Number of negative points
class_weights = {
    -1: 1,                                                                                # r (negative points)
    1: n_neg / n_pos                                                                      # r+ (positive points)
}
svm_2 = SVC(kernel='linear', probability=True, class_weight=class_weights, random_state=42)# Train an SVM model with class weights
svm_2.fit(X_train, y_train)
y_pred_2= svm_2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_2)
print(f"Model Accuracy: {accuracy:.4f}")                                                  # for test set
print("\nClassification Report:")
print(classification_report(y_test, y_pred_2))
#------------------------------------- Extract weight vector (w) and bias (b)-------------------------------------------
w = svm_2.coef_[0]   
b = svm_2.intercept_[0]  
print("Extracted Model Parameters:")
print(f"Weight Vector (w): {w}")
print(f"Bias (b): {b}")
#------------------------------------- Generate the new unseen instance------------------------------------------------------
np.random.seed(42)
num_instances_test = 1 
n_cities_test = 100  
distance_matrices_test = []
for instance in range(num_instances_test):
    distance_matrix_test = np.random.rand(n_cities_test, n_cities_test) * 100      
    np.fill_diagonal(distance_matrix_test, 0)
    distance_matrix_test = np.round(distance_matrix_test, 2)
    distance_matrices_test.append((instance + 1, distance_matrix_test)) 
print("\n--- Distance Matrix for new unseen instance ---") 
print(print(distance_matrices_test[0][1]))
#-----------------------------------Compute f1, f2, f3 scores for all edges in new unseen instance----------------------------------------------
f1_scores, f2_scores, f3_scores, f4_scores = {}, {}, {}, {}
for num_instances_test, distance_matrix_test in distance_matrices_test:
    f1, f2, f3, f4 = compute_graph_features(num_instances_test, distance_matrix_test)
    f1_scores.update(f1)
    f2_scores.update(f2)
    f3_scores.update(f3)
    f4_scores.update(f4)
f1_df = pd.DataFrame(f1_scores.items(), columns=["Instance_Edge", "f1"])
f2_df = pd.DataFrame(f2_scores.items(), columns=["Instance_Edge", "f2"])
f3_df = pd.DataFrame(f3_scores.items(), columns=["Instance_Edge", "f3"])
f4_df = pd.DataFrame(f4_scores.items(), columns=["Instance_Edge", "f4"])
for df in [f1_df, f2_df, f3_df, f4_df]:                                                     # Extract instance and edge columns
    df[['Instance', 'Edge']] = pd.DataFrame(df['Instance_Edge'].tolist(), index=df.index)
    df.drop(columns=["Instance_Edge"], inplace=True)
final_features_df = f1_df.merge(f2_df, on=['Instance', 'Edge'])
final_features_df = final_features_df.merge(f3_df, on=['Instance', 'Edge'])
final_features_df = final_features_df.merge(f4_df, on=['Instance', 'Edge'])
f_test = final_features_df[['f1', 'f2', 'f3', 'f4']].values 
w_column = w.reshape(-1, 1)  
#-------------------------------------Compute Z scalars------------------------------------------------------
z_values = np.dot(f_test,w_column) + b                                                      # (z = f_test * w^T + b) 
#------------------------------------- Compute predicted probability values for all edges in test instance-------------------------------------
p_values = 1 / (1 + np.exp(-z_values))                                                      # sigmoid function (p = 1 / (1 + e^(-z)))
z_values = z_values.flatten()
p_values = p_values.flatten()
p_values_df = pd.DataFrame({
    'Instance': final_features_df['Instance'].values,  
    'Edge': final_features_df['Edge'].values,  
    'p': p_values  
})
z_p_df = pd.DataFrame({                                                                     # Create a DataFrame with computed z and p values
    'Instance': final_features_df['Instance'],
    'Edge': final_features_df['Edge'],
    'z': z_values,
    'p': p_values
})
final_features_df = final_features_df.drop(columns=['z', 'p'], errors='ignore')            # Ensure no duplicate z and p columns before merging
final_combined_df = final_features_df[['Instance', 'Edge', 'f1', 'f2', 'f3', 'f4']].merge(
    z_p_df, on=['Instance', 'Edge'], how='left'
)
final_combined_df.head()
#----------------------------------------- Classical ACO algorithm for TSP-------------------------------------------------
random.seed(42)
np.random.seed(42)
class AntColony(object):
    def __init__(self, distances, n_ants, n_best, iterations, decay, alpha=1, beta=1):
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants                                                   # Number of ants running per iteration                                    
        self.n_best = n_best                                                   # Number of best ants who deposit pheromone
        self.iterations = iterations
        self.decay = decay                                                     # Rate it which pheromone decays
        self.alpha = alpha                                                     # exponenet on pheromone
        self.beta = beta                                                       # exponent on distance
    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            print (shortest_path)
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path
    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]
    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist
    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths
    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))   
        return path
    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0        
        dist[dist == 0] = np.inf 
        probability = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)
        norm_probability = probability / probability.sum()
        move = np_choice(self.all_inds, 1, p=norm_probability)[0]
        return move    
ant_colony = AntColony(distance_matrix_test, 50, 1, 100, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()
print ("shorted_path: {}".format(shortest_path))
#---------------------------------------------Run time---------------------------------------------------------------------------

start_time_classical_aco = time.time()                                      # Start time
ant_colony = AntColony(distance_matrix_test, 50, 1, 100, 0.95, alpha=1, beta=1)
shortest_path_classical_aco = ant_colony.run()
end_time_classical_aco = time.time()  # End time
runtime_classical_aco = end_time_classical_aco - start_time_classical_aco  # Calculate runtime
print(f"Classical ACO Runtime: {runtime_classical_aco:.4f} seconds")

#-------------------------------------------- Ml-ACO algorithm with extracting p_values-------------------------------------------
random.seed(42)
np.random.seed(42)
class AntColony_ML(object):
    def __init__(self, distances, p_values_df, n_ants, n_best, iterations, decay, alpha=1, beta=1):
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape)/len(distances)
        np.fill_diagonal(self.pheromone, 0)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.iterations = iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.n_cities = len(distances)
        self.p_values_df = p_values_df 
        self.p_values = self._extract_p_values()        
    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            print (shortest_path)
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path    
    def _extract_p_values(self):
        """Convert DataFrame p_values into a matrix format"""
        p_matrix = np.ones((self.n_cities, self.n_cities))  
        np.fill_diagonal(p_matrix, 0)       
        for _, row in self.p_values_df.iterrows():
            i, j = row['Edge']
            if i >= self.n_cities or j >= self.n_cities:  # Skip invalid edges
                continue
            p_matrix[i, j] = row['p']        
        return p_matrix
    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]
    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist
    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths
    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev],self.p_values[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started    
        return path
    def pick_move(self, pheromone, dist,p_values, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0        
        probability = (pheromone ** self.alpha) * ((p_values * (1.0 / dist)) ** self.beta)
        norm_probability= probability / probability.sum()
        move = np_choice(self.all_inds, 1, p=norm_probability)[0]
        return move    
ant_colony_ML = AntColony_ML(distance_matrix_test,p_values_df, 50, 1, 100, 0.95, alpha=1, beta=1)
shortest_path = ant_colony_ML.run()
print ("shorted_path: {}".format(shortest_path))
#---------------------------------------------Run time---------------------------------------------------------------------------
start_time_ml_aco = time.time()  # Start time
ant_colony_ML = AntColony_ML(distance_matrix_test, p_values_df, 50, 1, 100, 0.95, alpha=1, beta=1)
shortest_path_ml_aco = ant_colony_ML.run()
end_time_ml_aco = time.time()  # End time
runtime_ml_aco = end_time_ml_aco - start_time_ml_aco  # Calculate runtime
print(f"ML-enhanced ACO Runtime: {runtime_ml_aco:.4f} seconds")


Decision Variable Values (x[i][j])
x[0][1] = 0.00
x[0][2] = 0.00
x[0][3] = 0.00
x[0][4] = 0.00
x[0][5] = 0.00
x[0][6] = 0.00
x[0][7] = 0.00
x[0][8] = 0.00
x[0][9] = 0.00
x[0][10] = 0.00
x[0][11] = 0.00
x[0][12] = 0.00
x[0][13] = 0.00
x[0][14] = 1.00
x[0][15] = 0.00
x[0][16] = 0.00
x[0][17] = 0.00
x[0][18] = 0.00
x[0][19] = 0.00
x[0][20] = 0.00
x[0][21] = 0.00
x[0][22] = 0.00
x[0][23] = 0.00
x[0][24] = 0.00
x[0][25] = 0.00
x[0][26] = 0.00
x[0][27] = 0.00
x[0][28] = 0.00
x[0][29] = 0.00
x[1][0] = 0.00
x[1][2] = 0.00
x[1][3] = 0.00
x[1][4] = 0.00
x[1][5] = 0.00
x[1][6] = 0.00
x[1][7] = 0.00
x[1][8] = 0.00
x[1][9] = 0.00
x[1][10] = 0.00
x[1][11] = 0.00
x[1][12] = 0.00
x[1][13] = 0.00
x[1][14] = 0.00
x[1][15] = 0.00
x[1][16] = 0.00
x[1][17] = 0.00
x[1][18] = 0.00
x[1][19] = 0.00
x[1][20] = 1.00
x[1][21] = 0.00
x[1][22] = 0.00
x[1][23] = 0.00
x[1][24] = 0.00
x[1][25] = 0.00
x[1][26] = 0.00
x[1][27] = 0.00
x[1][28] = 0.00
x[1][29] = 0.00
x[2][0] = 0.00
x[2][1] = 0.00
x[2][3] = 0.00
x[2][4] =

In [2]:
final_combined_df.head()

,Instance,Edge,f1,f2,f3,f4,z,p
0,1,"(0, 1)",0.963,0.953,0.477,0.471,-11.310856,0.000012
1,1,"(0, 2)",0.742,0.733,0.257,0.244,-8.160403,0.000286
2,1,"(0, 3)",0.607,0.604,0.125,0.101,-6.230017,0.001966
3,1,"(0, 4)",0.158,0.157,-0.321,-0.316,-0.093724,0.476586
4,1,"(0, 5)",0.158,0.157,-0.322,-0.342,0.061923,0.515476
